In [91]:
# First, install the autoreload extension
%load_ext autoreload
# Set autoreload to automatically reload modules
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from algorithm import CR_0_20240118 as CR
from algorithm import CR_baseline_0_20240118 as CR_baseline



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
timed_data = pd.read_csv('../../data/compas/preprocessed/cox-parsed_7214rows_with_labels_sorted_by_dates.csv')
# get distribution of compas_screening_date
timed_data['compas_screening_date'] = pd.to_datetime(timed_data['compas_screening_date'])
alpha = 0.5
threshold = 0.3

In [103]:
########################################### 0 - 599
sub = timed_data[:600]
# FPR for caucasian:
FP_Cau = 0
TN_Cau = 0
def func_White(row):
    global FP_Cau
    global TN_Cau
    if row['race'] == "Caucasian":
        if row["predicted"] == 1 and row["ground_truth"] == 0:
            FP_Cau += 1
        elif row["predicted"] == 0 and row["ground_truth"] == 0:
            TN_Cau += 1
sub.apply(func_White, axis=1)

# FPR for Black:
FP_Black = 0
TN_Black = 0
def func_Black(row):
    global FP_Black
    global TN_Black
    if row['race'] == "African-American":
        if row["predicted"] == 1 and row["ground_truth"] == 0:
            FP_Black += 1
        elif row["predicted"] == 0 and row["ground_truth"] == 0:
            TN_Black += 1
sub.apply(func_Black, axis=1)
print(TN_Cau, TN_Black)
print(FP_Cau, FP_Black)
print(FP_Cau/(FP_Cau+TN_Cau), FP_Black/(FP_Black+TN_Black))


97 79
41 96
0.2971014492753623 0.5485714285714286


In [115]:
########################################### 0 - 599
sub = timed_data[:600]
# FPR for caucasian:
FP_Cau = 0
TN_Cau = 0
def func_White(row):
    global FP_Cau
    global TN_Cau
    if row['race'] == "Caucasian":
        if row["predicted"] == 1 and row["ground_truth"] == 0:
            FP_Cau += 1
        elif row["predicted"] == 0 and row["ground_truth"] == 0:
            TN_Cau += 1
sub.apply(func_White, axis=1)

# FPR for Black:
FP_Black = 0
TN_Black = 0
def func_Black(row):
    global FP_Black
    global TN_Black
    if row['race'] == "African-American":
        if row["predicted"] == 1 and row["ground_truth"] == 0:
            FP_Black += 1
        elif row["predicted"] == 0 and row["ground_truth"] == 0:
            TN_Black += 1
sub.apply(func_Black, axis=1)
print(TN_Cau, TN_Black)
print(FP_Cau, FP_Black)
print(FP_Cau/(FP_Cau+TN_Cau), FP_Black/(FP_Black+TN_Black))

######################## 600 ##############

FP_Black *= alpha
TN_Black *= alpha
FP_Cau *= alpha
TN_Cau *= alpha

sub = timed_data[600:631]
sub.apply(func_White, axis=1)
sub.apply(func_Black, axis=1)

print(TN_Cau, TN_Black)
print(FP_Cau, FP_Black)
print(FP_Cau/(FP_Cau+TN_Cau), FP_Black/(FP_Black+TN_Black))

######################## 601 - 616 ##############
# 
# 
# sub = timed_data[601:602]
# sub.apply(func_White, axis=1)
# sub.apply(func_Black, axis=1)
# print(FP_Cau, TN_Cau, FP_Cau/(FP_Cau+TN_Cau))
# print("white delta = {}".format(abs(threshold * TN_Cau - (1-threshold) * FP_Cau)))
# print(FP_Black, TN_Black, FP_Black/(FP_Black+TN_Black))
# print("black delta = {}".format(abs(threshold * TN_Black - (1-threshold) * FP_Black)))

97 79
41 96
0.2971014492753623 0.5485714285714286
51.5 45.5
22.5 54.0
0.30405405405405406 0.542713567839196


In [118]:
print(0.3, 97, 0.3*97*0.5)

0.3 97 14.549999999999999


In [119]:
print(0.3, 97, 0.3*97*0.5 >= 14.55)

0.3 97 False
